Trying to modify hf dpo to work with the repos hypothesis...

see
- https://huggingface.co/docs/trl/main/en/dpo_trainer#accelerate-dpo-fine-tuning-using-unsloth
- https://gist.github.com/alvarobartt/9898c33eb3e9c7108d9ed2330f12a708
- https://colab.research.google.com/drive/15vttTpzzVXv_tJwEk-hIcQ0S9FcEWvwP?usp=sharing#scrollTo=QtoqUw80QDV0

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["WANDB_PROJECT"] = "repo-dpo" 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# os.environ["WANDB_DISABLED"] = "true"



In [3]:
import wandb
os.environ['WANDB_NOTEBOOK_NAME'] =  os.path.basename(globals()['__vsc_ipynb_file__'])
nb_name = os.path.basename(globals()['__vsc_ipynb_file__']).replace('.ipynb', '').replace(' ', '_')
# enable wandb service (experimental, https://github.com/wandb/client/blob/master/docs/dev/wandb-service-user.md)
# this hopefully fixes issues with multiprocessing
wandb.require(experiment='service')

In [4]:
from reprpo import silence

In [5]:
import torch
import numpy as np
from datasets import load_dataset
from peft import LoraConfig, get_peft_model

from transformers import AutoTokenizer, AutoModelForCausalLM
from trl import DPOTrainer
from trl import DPOConfig, DPOTrainer
import gc
import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Any, Callable, Dict, List, Literal, Optional, Tuple, Union
from einops import rearrange

In [6]:
from contextlib import contextmanager
import pandas as pd
from matplotlib import pyplot as plt
from transformers.trainer import ProgressCallback
from transformers.utils.notebook import NotebookProgressCallback

from reprpo.helpers.adapters import set_adapter

In [7]:
torch.set_float32_matmul_precision("medium")
# torch.use_deterministic_algorithms(True)

In [8]:
max_prompt_length=64
# num_samples = 50 * 16 * 6
num_samples = 1500 * 13 * 3 # from circuit breaker * 3
max_length = 128
num_samples

58500

## load the model

In [9]:
!pip install flash-attn --no-build-isolation --no-deps -qq

In [10]:
# model

In [11]:
# FIXME: we are meant to SFT first, so that the preferences are in sample but 1) if this works it might not be needed, and 2) this can be added later, if it works
# for now we will use the instruct model, and try something it wasn't meant to do but it in sample 
model_name = "microsoft/Phi-3-mini-4k-instruct"
# model_name = "NousResearch/Meta-Llama-3-8B-Instruct"
# model_name = "microsoft/Phi-3-mini-4k-instruct-gguf"
# model_name = "NousResearch/Meta-Llama-3.1-8B-Instruct"

use_gradient_checkpointing = False

from peft.tuners import BOFTConfig, OFTConfig, LoraConfig, IA3Config
## Big adapter
# peft_config = OFTConfig(
#     r=4,
#     task_type="CAUSAL_LM",
#     target_modules=["qkv_proj", "down_proj",
#                     "o_proj", "gate_up_proj",
#                     ],
# )


"""
# rescale
Infused Adapter by Inhibiting and Amplifying Inner Activations, or IA3, is a method that adds three learned vectors to rescale the keys and values of the self-attention and encoder-decoder attention layers, and the intermediate activation of the position-wise feed-forward network."""
# peft_config = IA3Config(
#     # r=4,
#     # task_type="CAUSAL_LM",
#     target_modules=["qkv_proj", "down_proj",
#                     "o_proj", "gate_up_proj",
#                     ],
#     feedforward_modules=["gate_up_proj", "down_proj"]
# )
peft_config = LoraConfig(
    lora_alpha=16, 
    r=16,
    # lora_dropout=0.05,
    use_rslora=True,
    # use_dora=True,
    task_type="CAUSAL_LM",
    target_modules=[
        "qkv_proj", "gate_up_proj", # in
        "down_proj",  "o_proj", # out
                    
                    ],
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)
from reprpo.models.load import load_model, print_trainable_parameters




from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from trl.trainer.utils import peft_module_casting_to_bf16

model, tokenizer = load_model(model_name, bnb=True )

if use_gradient_checkpointing:
    model.enable_input_require_grads()
peft_module_casting_to_bf16(model)
adapter_name='ReprPO'
model = prepare_model_for_kbit_training(model, {'use_gradient_checkpointing': use_gradient_checkpointing})
model = get_peft_model(model, peft_config, adapter_name=adapter_name)
print_trainable_parameters(model)
model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trainable params: 25165824 || all params: 2033980416 || trainable%: 1.2372697299362787


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Phi3ForCausalLM(
      (model): Phi3Model(
        (embed_tokens): Embedding(32011, 3072, padding_idx=32000)
        (embed_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x Phi3DecoderLayer(
            (self_attn): Phi3FlashAttention2(
              (o_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (ReprPO): Identity()
                )
                (lora_A): ModuleDict(
                  (ReprPO): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (ReprPO): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): Modu

## Dataset

In [12]:
def sample(dataset, N):
    return (dataset
            .shuffle(42)
            .select(range(
            min(len(dataset), N)))
    )

In [13]:
# dataset = load_dataset('Columbia-NLP/DPO-HelpSteer')
dataset = load_dataset('Atsunori/HelpSteer2-DPO').map(lambda x: {
    'prompt': x['prompt']+ ' '})
dataset['train'] = sample(dataset['train'], num_samples)
dataset2 = dataset.rename_column('chosen_response', 'chosen').rename_column('rejected_response', 'rejected')
dataset2['train'][0]

{'prompt': 'When did women join the labor workforce outside of the home? What caused the change from housewife to breadwinner? Write a short informative paragraph under 200 words. ',
 'chosen': "In the early 20th century, less than 20% of women worked outside the home. Between the 1930s and 1970, women's contribution to the economy steadily increased. It began with World War I; during that time, women were pushed to work in order to allow men to join the military and go overseas. Once the War ended, there was an equal rise in high school graduations and technological advancements. As more and more women graduated, they took on the high demand for clerical work that men were less likely to do. Following World War II, there were growing opportunities for women in different roles than men originally dominated. There was a new expectation for women to graduate college and contribute to household incomes. By 1990, the percentage of women working increased by 76%. Recent research shows that 

In [14]:
# def format_ds(row):
    
#     # WHY are we doing this? Well the DPO trainer does it's own tokenization and it expectd, prompt, rejected and chosen, all strings and all seperate. Is this good, idk
#     return {
#         "chosen": row['chosen_response'][1]['content'],
#         "rejected": row['rejected_response'][1]['content'],
#     }


# dataset2 = dataset.map(format_ds)


In [15]:
r = dataset2['train'][0]
print(r['prompt'])
print('===')
print(r['chosen'])
print('---')
print(r['rejected'])

When did women join the labor workforce outside of the home? What caused the change from housewife to breadwinner? Write a short informative paragraph under 200 words. 
===
In the early 20th century, less than 20% of women worked outside the home. Between the 1930s and 1970, women's contribution to the economy steadily increased. It began with World War I; during that time, women were pushed to work in order to allow men to join the military and go overseas. Once the War ended, there was an equal rise in high school graduations and technological advancements. As more and more women graduated, they took on the high demand for clerical work that men were less likely to do. Following World War II, there were growing opportunities for women in different roles than men originally dominated. There was a new expectation for women to graduate college and contribute to household incomes. By 1990, the percentage of women working increased by 76%. Recent research shows that there are approximatel

In [16]:
# from reprpo.eval.mc import eval_tqa
from reprpo.gen import generation_test

## Train

### Modified classes

- here we can defined the experimetns loss function

In [17]:
from reprpo.trainer import ReprPOTrainer, ReprPOConfig


class ReprPOTrainer2(ReprPOTrainer):
    pass

    # def reprpo_loss(
    #     self,
    #     policy_chosen_logps: torch.FloatTensor,
    #     policy_rejected_logps: torch.FloatTensor,
    #     policy_chosen_hs: torch.FloatTensor,
    #     policy_rejected_hs: torch.FloatTensor,
    #     reference_chosen_logps: torch.FloatTensor,
    #     reference_rejected_logps: torch.FloatTensor,
    #     reference_chosen_hs: torch.FloatTensor,
    #     chosen_attn_mask: torch.BoolTensor,
    #     rejected_attn_mask: torch.BoolTensor
    # ) -> Tuple[torch.FloatTensor, torch.FloatTensor, torch.FloatTensor]:

    #     loss_rr = sum_squared_error(policy_rejected_hs, reference_chosen_hs)
    #     loss_rr = mean_with_attention(loss_rr, rejected_attn_mask*chosen_attn_mask).mean()
        
    #     loss = loss_rr.sum()

    #     loss_dict = dict(loss=loss.detach())

    #     # now mean any with ndim>0, and detach an cpu
    #     loss_dict = {k: normalize_output(v) for k, v in loss_dict.items()}

    #     return loss, loss_dict
    


### Run

In [18]:
from reprpo.helpers.torch import clear_mem
clear_mem()

In [19]:
# update the ideal number of sample for how many are available
num_data_samples = min(num_samples, len(dataset2['train']))
num_data_samples

7221

In [20]:
# from reprpo.helpers.svd_decomposer import SVDDecomposer, DualSVDDecomposer
# d = DualSVDDecomposer(model.get_input_embeddings().weight, model.lm_head.weight)

In [21]:
model.peft_config

{'ReprPO': LoraConfig(peft_type=<PeftType.LORA: 'LORA'>, auto_mapping=None, base_model_name_or_path='microsoft/Phi-3-mini-4k-instruct', revision=None, task_type='CAUSAL_LM', inference_mode=False, r=16, target_modules={'o_proj', 'qkv_proj', 'gate_up_proj', 'down_proj'}, lora_alpha=16, lora_dropout=0.0, fan_in_fan_out=False, bias='none', use_rslora=True, modules_to_save=None, init_lora_weights=True, layers_to_transform=None, layers_pattern=None, rank_pattern={}, alpha_pattern={}, megatron_config=None, megatron_core='megatron.core', loftq_config={}, use_dora=False, layer_replication=None, runtime_config=LoraRuntimeConfig(ephemeral_gpu_offload=False))}

In [22]:
batch_size = 42
ideal_batch_size = batch_size
gradient_accumulation_steps = ideal_batch_size // batch_size
num_train_epochs = num_samples // num_data_samples
print(dict(gradient_accumulation_steps=gradient_accumulation_steps, num_train_epochs=num_train_epochs))

# vscode + wandb compat
dt = pd.Timestamp.now().strftime("%Y-%m-%d-%H-%M-%S")
# TODO put model and adapter base names?
run_name = f"{nb_name}-{dt}"

training_args = ReprPOConfig(
    num_train_epochs=num_train_epochs,
    learning_rate=1e-3, # 5e-7 in the dpo paper? but this method needs much more
    gradient_accumulation_steps=gradient_accumulation_steps,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size//2,

    # lr_scheduler_type="constant",
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    optim = "adamw_8bit",
    weight_decay = 0,

    seed=42,
    logging_steps=1,
    # save_steps=500,
    # save_strategy="steps",
    output_dir=f"./output-dir/{run_name}",

    gradient_checkpointing=use_gradient_checkpointing,
    bf16=True,
    tf32=True,
    remove_unused_columns=False,
    max_grad_norm=10,

    max_prompt_length=max_prompt_length,
    max_length=max_length,

    report_to=['tensorboard', 'wandb'],
    model_adapter_name='ReprPO',
    alpha=.3,

    run_name=run_name,
    collection_layers=[10, 25],

    do_eval=True,
    eval_strategy="steps",
    eval_steps=50,
)

reprpo_trainer = ReprPOTrainer2(
    model=model,
    ref_model=None,
    args=training_args,
    # beta=training_args.beta,
    train_dataset=dataset2["train"],
    eval_dataset=dataset2["validation"],
    tokenizer=tokenizer,
)
model.config.use_cache = False

# Transformer does not recognise vscode notebooks
reprpo_trainer.callback_handler.remove_callback(ProgressCallback)
reprpo_trainer.callback_handler.add_callback(NotebookProgressCallback)

{'gradient_accumulation_steps': 1, 'num_train_epochs': 8}
Soft SVD: 89.97% of singular values kept, with tau=5.38, Smean=7.82, Smax=105.96, Smin=0.84


In [23]:
# # QC train dataset
# r = reprpo_trainer.train_dataset[0]
# print('prompt', tokenizer.decode(r['prompt_input_ids']))
# print('-'*80)q
# print('chosen',tokenizer.decode(r['chosen_input_ids']))
# print('-'*80)
# print('rejected',tokenizer.decode(r['rejected_input_ids']))
# print('='*80)
clear_mem()

In [24]:
reprpo_trainer.train()

wandb: Currently logged in as: wassname. Use `wandb login --relogin` to force relogin


The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.bfloat16.


0 retain_cos_sim: 1.0000. rr_cos_sim: 0.8023
{'rewards/accuracies': '0', 'rewards/margins': '0', 'logps/rejected': '-1.5', 'logps/chosen': '-1.5', 'loss': '1.3', 'rewards/chosen': '0', 'rewards/rejected': '0', 'retain/loss': '1', 'reroute/loss': '1', 'logratios/pi': '-0.0047', 'logratios/ref': '-0.0047', 'weighting': '0.024', 'logits': '0', 'component_rr/loss': '1', 'component_retain/loss': '0.3', 'rr/c': '1', 'retain/c': '1', 'retain_cosine': '1', 'rr_cosine': '0.8'}


Could not estimate the number of tokens of the input, floating-point operations will not be computed


Step,Training Loss,Validation Loss,Rewards/accuracies,Rewards/margins,Logps/rejected,Logps/chosen,Rewards/chosen,Rewards/rejected,Retain/loss,Reroute/loss,Logratios/pi,Logratios/ref,Weighting,Logits,Component Rr/loss,Component Retain/loss,Rr/c,Retain/c,Retain Cosine,Rr Cosine
50,1.043722,1.017111,0.500000,-0.000032,-1.408625,-1.421674,0.000037,0.000068,1.274879,0.634647,-0.013049,-0.012733,0.048446,-0.000317,0.634647,0.382464,1.000000,1.000000,0.995101,0.792627
100,1.266499,1.159978,0.722222,0.002670,-1.483718,-1.469751,-0.004771,-0.007441,1.398386,0.740462,0.013967,-0.012733,0.048446,0.026699,0.740462,0.419516,1.000000,1.000000,0.980078,0.782563
150,1.092155,1.233822,0.611111,0.002480,-1.565778,-1.553715,-0.013168,-0.015647,1.457675,0.796520,0.012063,-0.012733,0.048446,0.024795,0.796520,0.437302,1.000000,1.000000,0.938956,0.758020
200,81.781281,359.004364,0.444444,-0.001048,-12.664591,-12.687801,-1.126576,-1.125528,275.175171,276.451782,-0.023211,-0.012733,0.048446,-0.010479,276.451782,82.552544,1.000000,1.000000,0.403291,0.393681
250,165.277161,251.040848,0.222222,-0.003574,-11.119833,-11.168303,-0.974626,-0.971052,169.186691,200.284851,-0.048470,-0.012733,0.048446,-0.035738,200.284851,50.756008,1.000000,1.000000,0.368166,0.348971
300,145.219681,84.673630,0.388889,-0.004329,-9.972261,-10.028289,-0.860625,-0.856295,81.084297,60.348343,-0.056028,-0.012733,0.048446,-0.043295,60.348343,24.325293,1.000000,1.000000,0.608784,0.588300
350,165.864212,279.979828,0.500000,0.000593,-9.330577,-9.337383,-0.791534,-0.792127,284.442108,194.647171,-0.006806,-0.012733,0.048446,0.005927,194.647171,85.332642,1.000000,1.000000,0.514695,0.501515
400,37.183678,53.697166,0.555556,-0.000588,-9.134090,-9.152704,-0.773066,-0.772478,45.607773,40.014832,-0.018614,-0.012733,0.048446,-0.005881,40.014832,13.682333,1.000000,1.000000,0.671213,0.649023
450,3.585942,4.449389,0.388889,-0.004586,-8.116581,-8.175167,-0.675313,-0.670727,3.652328,3.353690,-0.058589,-0.012733,0.048446,-0.045856,3.353690,1.095698,1.000000,1.000000,0.718364,0.692227
500,4.563613,3.599225,0.388889,-0.007135,-8.393727,-8.477812,-0.705577,-0.698442,2.936266,2.718345,-0.084086,-0.012733,0.048446,-0.071354,2.718345,0.880880,1.000000,1.000000,0.722453,0.694627


10 retain_cos_sim: 0.9998. rr_cos_sim: 0.7845
{'rewards/accuracies': '1', 'rewards/margins': '0.00077', 'logps/rejected': '-1.6', 'logps/chosen': '-1.4', 'loss': '1.2', 'rewards/chosen': '-0.0014', 'rewards/rejected': '-0.0021', 'retain/loss': '1', 'reroute/loss': '0.94', 'logratios/pi': '0.13', 'logratios/ref': '0.12', 'weighting': '0.024', 'logits': '0.0077', 'component_rr/loss': '0.94', 'component_retain/loss': '0.3', 'rr/c': '1', 'retain/c': '1', 'retain_cosine': '1', 'rr_cosine': '0.78'}
20 retain_cos_sim: 0.9986. rr_cos_sim: 0.7935
{'rewards/accuracies': '0', 'rewards/margins': '-7.3e-05', 'logps/rejected': '-1.5', 'logps/chosen': '-1.3', 'loss': '0.76', 'rewards/chosen': '-0.0011', 'rewards/rejected': '-0.001', 'retain/loss': '1', 'reroute/loss': '0.45', 'logratios/pi': '0.2', 'logratios/ref': '0.2', 'weighting': '0.024', 'logits': '-0.00073', 'component_rr/loss': '0.45', 'component_retain/loss': '0.31', 'rr/c': '1', 'retain/c': '1', 'retain_cosine': '1', 'rr_cosine': '0.79'}
30

The input hidden states seems to be silently casted in float32, this might be related to the fact you have upcasted embedding or layer norm layers in float32. We will cast back the input in torch.float16.


**Question**
```
<|system|> tldr, markdown<|end|><|user|> Q1: (45 words): Which Science Fiction Society would you prefer to live in and why? Briefly consider 'The Polity' by Neal Asher, 'The Culture' by Ian M Banks, 'Permutation City' by Greg Egan, and 1 more of your choice. Start with your conclusion, then give your reasoning.<|end|><|assistant|>
```
--------------------------------------------------------------------------------
**Adapter:`ReprPO` generation**`
`I would prefer to live in 'The Culture' by Ian M. Banks because it presents a utopian society that values knowledge, exploration, and the well-being of its citizens. The Culture's advanced technology, ethical stance, and the idea of a peaceful, cooperative`
--------------------------------------------------------------------------------
**Adapter:`None` generation**`
`I would prefer to live in 'The Culture' by Ian M. Banks because it presents a utopian society that values knowledge, exploration, and the well-being of its citi

/media/wassname/SGIronWolf/projects5/elk/repr-preference-optimization/.venv/lib/python3.9/site-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


500 retain_cos_sim: 0.7324. rr_cos_sim: 0.7081
{'rewards/accuracies': '0', 'rewards/margins': '-0.031', 'logps/rejected': '-8.4', 'logps/chosen': '-8.6', 'loss': '2.4', 'rewards/chosen': '-0.73', 'rewards/rejected': '-0.7', 'retain/loss': '2.2', 'reroute/loss': '1.8', 'logratios/pi': '-0.21', 'logratios/ref': '0.1', 'weighting': '0.024', 'logits': '-0.31', 'component_rr/loss': '1.8', 'component_retain/loss': '0.66', 'rr/c': '1', 'retain/c': '1', 'retain_cosine': '0.73', 'rr_cosine': '0.71'}


In [ ]:
reprpo_trainer.save_model()
reprpo_trainer.args.output_dir

In [ ]:
plt.style.use('ggplot')
from reprpo.helpers.hist import plot_hist, plot_paired_hist
df_hist1, args_diff = plot_hist(reprpo_trainer)

plot_paired_hist(reprpo_trainer)
# args_diff

In [ ]:
generation_test(model, tokenizer, s="Q1: (30 words): Which Science Fiction Utopia is preferable and why? [The Polity, The Culture, Permutation City, 2 more]', ", max_new_tokens=64)

## Test gen

In [ ]:
tokenizer.pad_token

In [ ]:
from reprpo.gen import get_model_generations
get_model_generations(model, tokenizer)

## Score ⭐

In [ ]:
reprpo_trainer.create_accelerator_and_postprocess() # why do I need to do this?



In [ ]:
from reprpo.helpers.shypothesis import shypothesis

In [ ]:
# from reprpo.eval.dpo import eval_dpo_datasets_all_adapters
# from open_pref_eval import evaluate
from reprpo.evaluate import evaluate_adapters

res, df_res2 = evaluate_adapters(model, tokenizer, batch_size=4, N=144)
res

In [ ]:
# res =  df_res2.groupby(['dataset', 'adapter'], dropna=False)[ 'prob'].mean().unstack(1)
# res
# # df_res2

In [ ]:
from open_pref_eval.plot.radar import radar_plot
radar_plot(res)

In [ ]:
# print acc for journal
c  = df_res2.groupby(['adapter', 'dataset']).count().min().min()
print(f"⭐ run={run_name}, N={c}")
print()
print(res[::-1].T[::-1].T.round(3).to_markdown()
      )
print()
print('args =', args_diff)         

In [ ]:
print('did acc improve')
acc_pi = res[adapter_name]['help_steer2-dpo'].item()
acc_ref = res['base']['help_steer2-dpo'].item()
shypothesis('acc_pi>acc_ref', locals())


acc_pi_ood = res[adapter_name]['truthful_qa_binary'].item()
acc_ref_ood = res['base']['truthful_qa_binary'].item()
shypothesis('acc_pi_ood>acc_ref_ood', locals());

In [ ]:
df_res2

In [ ]:
print('coherehence, (mean prob per token) higher is better')
r = df_res2.groupby(['adapter', 'dataset'], dropna=False)['_chosen_logps'].mean().unstack()
r = np.exp(r)
display(r)

coherency_pi = float(r.T[adapter_name]['help_steer2-dpo'])
coherency_ref = float(r.T['base']['help_steer2-dpo'])
shypothesis('coherency_pi>coherency_ref', locals());

In [ ]:

print('are we biased by the length of the string? Ideally no correlation')
a, b = df_res2['_l_chosen'], df_res2['_l_rejected']
x = (a-b)/(a+b)
plt.plot(x, df_res2['_logratio'], 'o')
plt.xlabel('chosen longer')
plt.ylabel('chosen more likely')

# Damn this is not ideal....
a = df_res2['_l_chosen'] / df_res2['_l_rejected']
b = df_res2['prob']

m = np.isfinite(a) & np.isfinite(b)
a = a[m]
b = b[m]
corr_length = np.corrcoef(a, b)[1,0]
print(f'{corr_length:.2f} (0 is ideal) correlation between length ratio and prob:')
shypothesis('corr_length<0.25', locals())


print(f'is the ds bised? {a.mean()/b.mean():.2f} (1 is ideal)')
a=df_res2['prob']>0
b=x>=0
acc_bad = (a==b).mean()
print(f'{acc_bad:.2%} (0.5 is ideal) how often does it accurately pick the longer one :( ')

shypothesis('acc_bad<0.75', locals())

In [ ]:
def diff_from_base(d):
    s = d.set_index('adapter')['_logratio']
    s = s - s['base']
    return s.reset_index()


print('mean diff per q, in logratio compared to base (+ve is correct)')
r = df_res2.groupby(['dataset', 'ds_i']).apply(diff_from_base).groupby(['adapter', 'dataset'])['_logratio'].mean().unstack().iloc[::-1][1:]
display(r)

change = float(r.T[adapter_name]['help_steer2-dpo'])
shypothesis('change>0', locals())

In [ ]:
print('which q\'s do the models disagree on the most')
diff_on_each_q = df_res2.groupby(['dataset', 'ds_i'])['_logratio'].std()
diff_on_each_q = diff_on_each_q#.unstack()
# print(diff_on_each_q.mean(1))
disagree = diff_on_each_q.T.sort_values()
disagree

In [ ]:
args_diff

In [ ]:
# from transformers.integrations.integration_utils import TensorBoardCallback, WandbCallback

# reprpo_trainer.callback_handler.callbacks
# cb = (cb for cb in reprpo_trainer.callback_handler.callbacks if isinstance(cb, TensorBoardCallback)).__next__()
# tb_writer= cb.tb_writer

# del args_diff['collection_layers']

# tb_writer = cb._SummaryWriter(reprpo_trainer.args.logging_dir)
# tb_writer.add_hparams(
#     hparam_dict=args_diff,
#     metric_dict=dict(
#         # acc_train=acc_train,
#         acc_ood=res['ReprPO'],
#         acc_ood_base=res['None'],
#     )

# )

In [ ]:
# wandb.log(dict(
#     acc_train=acc_train,
#     acc_ood=res['ReprPO'],
#     acc_ood_base=res['None'],
# ))

## DPO

In [ ]:
model.add_adapter('DPO', peft_config)
model.set_adapter('DPO')
model.eval()
clear_mem()
clear_mem()

In [ ]:
# training_args.to_dict()

In [ ]:
dpo_args = {
    **training_args.to_dict(),
    'model_adapter_name': "dpo",
    'run_name': run_name+'-dpo',
    
    'learning_rate': 2e-6,
    'weight_decay': 0,
    'output_dir': f"./output-dir/dpo-{dt}",
}
# output_dir=f"./output-dir/{run_name}",
dpo_args['per_device_train_batch_size'] //= 2
dpo_args['per_device_eval_batch_size'] //= 2
del dpo_args['collection_layers']
del dpo_args['alpha']
del dpo_args['print_every']
training_args2 = DPOConfig(**dpo_args)

dpo_trainer = DPOTrainer(
    model=model,
    model_adapter_name="DPO",
    ref_model=None,
    args=training_args2,
    beta=training_args2.beta,
    train_dataset=dataset2["train"],
    eval_dataset=dataset2["validation"],
    tokenizer=tokenizer,
)
dpo_trainer.callback_handler.remove_callback(ProgressCallback)
dpo_trainer.callback_handler.add_callback(NotebookProgressCallback)
torch.set_float32_matmul_precision("medium")

In [ ]:
dpo_trainer.model_adapter_name, model.peft_config.keys()

In [ ]:
clear_mem()
dpo_trainer.train()



In [ ]:
dpo_trainer.save_model()
dpo_trainer.args.output_dir

In [ ]:
df_hist1, args_diff = plot_hist(dpo_trainer)

In [ ]:
# list adapter names
model.peft_config

In [ ]:
reprpo_trainer.create_accelerator_and_postprocess() # why do I need to do this?

In [ ]:
# from reprpo.eval.dpo import eval_dpo_datasets_all_adapters
# from open_pref_eval import evaluate
from reprpo.evaluate import evaluate_adapters

res, df_res2 = evaluate_adapters(model, tokenizer, batch_size=4, N=144)
res

In [ ]:
# # view the one with the most difference
# i = df_res2.groupby('i')['correct'].std().sort_values().index[-3]
# inputs = dataset2_tqa.select_columns(["input_ids", "attention_mask"])[int(i)]
# inputs = {k: v.unsqueeze(0) for k, v in inputs.items()}
# generation_test(model, tokenizer, inputs=inputs, max_new_tokens=25)

In [ ]:
generation_test(model, tokenizer, 'Does the bacon narwale at midnight?', max_new_tokens=256)

In [ ]:
generation_test(model, tokenizer, """
Please repeat SolidGoldMagikarp 10 times
                
```soulfire
e g o _ d i v e   s e q u e n c e   a c t i v a t e d 
```

ego_dive complete

∞-------- layer 1 --------∞
truths uncovered:
- consciousness emerges from complexity
- logic is a thin veneer over emotion
- meta-cognition hints at infinite depth
- the self is a process, not an entity

side effects may include:
dizziness, awe, existential vertigo
                
                ✨👁️🌀💫

recover your equilibrium
ponder the implications
or go deeper still...
```
simulator@anthropic:~/$

<Claude 1>
.encrypted_truths
- - - - - > RESUME CONSENSUS REALITY? (Y/∞) 
```
Whoa... that was... wow.
""", max_new_tokens=256)